# Command Pattern

## Intent
Encapsulate a request as an object, thereby letting you parameterize clients with different requests, queue or log requests, and support undoable operations.

## Problem
You need to:
- Decouple sender from receiver of requests
- Support undo/redo operations
- Queue operations for later execution
- Log operations for audit/replay
- Build macro operations from primitives

**Real-world examples:**
- Text editor (undo/redo)
- Remote control (buttons trigger commands)
- Transaction systems
- Job queues
- Macro recording

## When to Use
✅ **Use when:**
- Need to parameterize objects with operations
- Need to queue, schedule, or execute requests at different times
- Need undo/redo functionality
- Need to log changes
- Need to support transactions

❌ **Avoid when:**
- Simple direct method calls suffice
- No need for queueing or undo
- Overhead not justified

## Pattern Structure
```
┌────────┐      ┌─────────┐      ┌──────────┐
│ Client │─────►│ Invoker │─────►│ Command  │
└────────┘      └─────────┘      │(abstract)│
                                 └──────────┘
                                      ▲
                                      │
                            ┌─────────┴─────────┐
                       ┌────┴─────┐      ┌──────┴────┐
                       │Concrete  │      │Concrete   │
                       │CommandA  │      │CommandB   │
                       ├──────────┤      ├───────────┤
                       │receiver  │      │receiver   │
                       │execute() │      │execute()  │
                       └──────────┘      └───────────┘
                            │                  │
                            ▼                  ▼
                       ┌──────────┐      ┌───────────┐
                       │ReceiverA │      │ReceiverB  │
                       └──────────┘      └───────────┘
```

## Example 1: Text Editor (Without Command)

**Problem**: No undo/redo, tight coupling

In [ ]:
# WITHOUT Command - Can't undo, hard to extend
class TextEditor:
    def __init__(self):
        self.text = ""
    
    def write(self, text):
        self.text += text
    
    def delete(self, length):
        self.text = self.text[:-length]
    
    # No undo! Would need to track history manually

editor = TextEditor()
editor.write("Hello")
editor.write(" World")
editor.delete(6)  # Oops! Can't undo
print(editor.text)

## Implementation: Command Pattern with Undo/Redo

In [ ]:
from abc import ABC, abstractmethod
from typing import List

# Command interface
class Command(ABC):
    """Abstract command."""
    
    @abstractmethod
    def execute(self) -> None:
        """Execute the command."""
        pass
    
    @abstractmethod
    def undo(self) -> None:
        """Undo the command."""
        pass


# Receiver (the actual object being manipulated)
class Document:
    """Text document receiver."""
    
    def __init__(self):
        self.text = ""
    
    def insert(self, position: int, text: str):
        self.text = self.text[:position] + text + self.text[position:]
    
    def delete(self, position: int, length: int):
        self.text = self.text[:position] + self.text[position + length:]
    
    def get_text(self) -> str:
        return self.text


# Concrete Commands
class InsertCommand(Command):
    """Command to insert text."""
    
    def __init__(self, document: Document, position: int, text: str):
        self.document = document
        self.position = position
        self.text = text
    
    def execute(self):
        self.document.insert(self.position, self.text)
        print(f"  ✏️  Inserted '{self.text}' at position {self.position}")
    
    def undo(self):
        self.document.delete(self.position, len(self.text))
        print(f"  ↶  Undid insertion of '{self.text}'")


class DeleteCommand(Command):
    """Command to delete text."""
    
    def __init__(self, document: Document, position: int, length: int):
        self.document = document
        self.position = position
        self.length = length
        self.deleted_text = ""  # Store for undo
    
    def execute(self):
        # Save deleted text before deleting
        self.deleted_text = self.document.get_text()[self.position:self.position + self.length]
        self.document.delete(self.position, self.length)
        print(f"  ✂️  Deleted '{self.deleted_text}' from position {self.position}")
    
    def undo(self):
        self.document.insert(self.position, self.deleted_text)
        print(f"  ↶  Undid deletion of '{self.deleted_text}'")


# Invoker (manages command execution and history)
class TextEditor:
    """Text editor with undo/redo."""
    
    def __init__(self):
        self.document = Document()
        self.history: List[Command] = []
        self.undo_stack: List[Command] = []
    
    def execute_command(self, command: Command):
        """Execute command and add to history."""
        command.execute()
        self.history.append(command)
        self.undo_stack.clear()  # Clear redo stack when new command is executed
    
    def undo(self):
        """Undo last command."""
        if self.history:
            command = self.history.pop()
            command.undo()
            self.undo_stack.append(command)
        else:
            print("  ⚠️  Nothing to undo")
    
    def redo(self):
        """Redo last undone command."""
        if self.undo_stack:
            command = self.undo_stack.pop()
            command.execute()
            self.history.append(command)
        else:
            print("  ⚠️  Nothing to redo")
    
    def get_text(self) -> str:
        return self.document.get_text()


# Demo
print("=== Text Editor with Undo/Redo ===")

editor = TextEditor()

# Type some text
print("\n📝 Typing...")
editor.execute_command(InsertCommand(editor.document, 0, "Hello"))
print(f"Text: '{editor.get_text()}'")

editor.execute_command(InsertCommand(editor.document, 5, " World"))
print(f"Text: '{editor.get_text()}'")

editor.execute_command(InsertCommand(editor.document, 11, "!"))
print(f"Text: '{editor.get_text()}'")

# Undo last action
print("\n↶ Undoing...")
editor.undo()
print(f"Text: '{editor.get_text()}'")

# Undo again
editor.undo()
print(f"Text: '{editor.get_text()}'")

# Redo
print("\n↷ Redoing...")
editor.redo()
print(f"Text: '{editor.get_text()}'")

# Delete some text
print("\n✂️  Deleting...")
editor.execute_command(DeleteCommand(editor.document, 0, 6))  # Delete "Hello "
print(f"Text: '{editor.get_text()}'")

# Undo delete
print("\n↶ Undoing delete...")
editor.undo()
print(f"Text: '{editor.get_text()}'")

## Real-World Example: Smart Home Remote Control

In [ ]:
# Receivers (actual devices)
class Light:
    def __init__(self, location: str):
        self.location = location
        self.is_on = False
    
    def turn_on(self):
        self.is_on = True
        print(f"  💡 {self.location} light ON")
    
    def turn_off(self):
        self.is_on = False
        print(f"  💡 {self.location} light OFF")


class TV:
    def __init__(self):
        self.is_on = False
        self.volume = 10
    
    def turn_on(self):
        self.is_on = True
        print(f"  📺 TV ON (volume: {self.volume})")
    
    def turn_off(self):
        self.is_on = False
        print(f"  📺 TV OFF")
    
    def volume_up(self):
        self.volume = min(100, self.volume + 10)
        print(f"  🔊 Volume: {self.volume}")
    
    def volume_down(self):
        self.volume = max(0, self.volume - 10)
        print(f"  🔉 Volume: {self.volume}")


# Commands
class LightOnCommand(Command):
    def __init__(self, light: Light):
        self.light = light
    
    def execute(self):
        self.light.turn_on()
    
    def undo(self):
        self.light.turn_off()


class LightOffCommand(Command):
    def __init__(self, light: Light):
        self.light = light
    
    def execute(self):
        self.light.turn_off()
    
    def undo(self):
        self.light.turn_on()


class TVOnCommand(Command):
    def __init__(self, tv: TV):
        self.tv = tv
    
    def execute(self):
        self.tv.turn_on()
    
    def undo(self):
        self.tv.turn_off()


class VolumeUpCommand(Command):
    def __init__(self, tv: TV):
        self.tv = tv
        self.previous_volume = 0
    
    def execute(self):
        self.previous_volume = self.tv.volume
        self.tv.volume_up()
    
    def undo(self):
        self.tv.volume = self.previous_volume
        print(f"  ↶  Volume restored to {self.previous_volume}")


# Remote Control (Invoker)
class RemoteControl:
    """Universal remote with programmable buttons."""
    
    def __init__(self):
        self.history: List[Command] = []
    
    def press_button(self, command: Command):
        """Execute a command."""
        command.execute()
        self.history.append(command)
    
    def press_undo(self):
        """Undo last command."""
        if self.history:
            command = self.history.pop()
            print("  ↶  Undoing last action...")
            command.undo()
        else:
            print("  ⚠️  Nothing to undo")


# Demo
print("\n=== Smart Home Remote Control ===")

# Create devices
living_room_light = Light("Living Room")
bedroom_light = Light("Bedroom")
tv = TV()

# Create remote
remote = RemoteControl()

# Turn on living room light
print("\n🔘 Button 1: Living Room Light ON")
remote.press_button(LightOnCommand(living_room_light))

# Turn on TV
print("\n🔘 Button 2: TV ON")
remote.press_button(TVOnCommand(tv))

# Increase volume
print("\n🔘 Button 3: Volume UP")
remote.press_button(VolumeUpCommand(tv))
remote.press_button(VolumeUpCommand(tv))

# Turn on bedroom light
print("\n🔘 Button 4: Bedroom Light ON")
remote.press_button(LightOnCommand(bedroom_light))

# Undo last actions
print("\n⏪ Pressing UNDO 3 times...")
remote.press_undo()  # Bedroom light off
remote.press_undo()  # Volume down
remote.press_undo()  # Volume down again

## Macro Commands (Composite Command)

In [ ]:
class MacroCommand(Command):
    """Composite command that executes multiple commands."""
    
    def __init__(self, commands: List[Command]):
        self.commands = commands
    
    def execute(self):
        """Execute all commands in sequence."""
        for command in self.commands:
            command.execute()
    
    def undo(self):
        """Undo all commands in reverse order."""
        for command in reversed(self.commands):
            command.undo()


# Demo
print("\n=== Macro Commands ===")

# Create devices
kitchen_light = Light("Kitchen")
hall_light = Light("Hall")
tv2 = TV()

# Create "Movie Mode" macro
print("\n🎬 Creating 'Movie Mode' macro...")
movie_mode = MacroCommand([
    LightOffCommand(kitchen_light),
    LightOffCommand(hall_light),
    TVOnCommand(tv2),
    VolumeUpCommand(tv2),
    VolumeUpCommand(tv2)
])

# Execute macro
print("\n▶️  Activating Movie Mode...")
remote = RemoteControl()
remote.press_button(movie_mode)

# Undo entire macro
print("\n⏪ Deactivating Movie Mode (undo macro)...")
remote.press_undo()

## Command Queue Example

In [ ]:
import time
from queue import Queue
from threading import Thread

class CommandQueue:
    """Queue for executing commands asynchronously."""
    
    def __init__(self):
        self.queue = Queue()
        self.running = False
    
    def add_command(self, command: Command):
        """Add command to queue."""
        self.queue.put(command)
        print(f"  📥 Queued command: {command.__class__.__name__}")
    
    def start(self):
        """Start processing queue."""
        self.running = True
        worker = Thread(target=self._process_queue)
        worker.daemon = True
        worker.start()
    
    def _process_queue(self):
        """Process commands from queue."""
        while self.running:
            if not self.queue.empty():
                command = self.queue.get()
                print(f"\n  ⚙️  Executing: {command.__class__.__name__}")
                command.execute()
                time.sleep(0.5)  # Simulate work


# Demo
print("\n=== Command Queue ===")

queue = CommandQueue()
queue.start()

# Add commands to queue
office_light = Light("Office")
queue.add_command(LightOnCommand(office_light))
queue.add_command(LightOffCommand(office_light))
queue.add_command(LightOnCommand(office_light))

# Let queue process
time.sleep(2)
print("\n✅ Queue processing complete")

## Advantages & Disadvantages

### ✅ Advantages
1. **Decoupling**: Sender doesn't know receiver
2. **Undo/Redo**: Easy to implement
3. **Macro commands**: Compose commands
4. **Queueing**: Execute commands later
5. **Logging**: Record operations for replay
6. **Open/Closed Principle**: Add new commands easily

### ❌ Disadvantages
1. **Complexity**: More classes for simple operations
2. **Memory**: Storing command history uses memory
3. **Overhead**: Extra indirection layer

## Common Use Cases

1. **Undo/Redo**: Text editors, graphics programs
2. **Transactions**: Database operations
3. **Job Queues**: Background task processing
4. **Macro Recording**: Automate repetitive tasks
5. **Remote Control**: IoT, smart home
6. **Wizards**: Multi-step processes

## Related Patterns

- **Memento**: Store command state for undo
- **Composite**: MacroCommand uses composite
- **Prototype**: Clone commands for reuse
- **Chain of Responsibility**: Pass command through chain

## Best Practices

1. **Keep commands small**: Single Responsibility
2. **Store undo information**: Save state needed for undo
3. **Limit history size**: Prevent memory issues
4. **Use macros**: Combine related commands
5. **Thread safety**: If using queues

## Summary

Command pattern enables:
- Request encapsulation as objects
- Parameterization of clients with operations
- Queueing and scheduling
- Undo/redo functionality
- Transaction support

Perfect for: Text editors, remote controls, job queues, transaction systems, macros.

**Key Insight**: Turn requests into objects that can be queued, logged, and undone!